# Betweenness centrality calculation for real data

Surfacic Networks
Barthelemy et al., PNAS Nexus 2025

In [ ]:
#%conda install -c conda-forge pandas
#conda activate ox
# generate x y random in disk(0,1) and z=1-x**2-y**2
# generate then rgg

import networkx as nx
import osmnx as ox
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import math
import csv
import os
import seaborn as sns
from colorama import Fore
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import pyplot as plt
from matplotlib import cm

mpl.rc('figure', max_open_warning = 0)

import itertools
from itertools import product
from itertools import combinations
from collections import Counter
import operator

from haversine import haversine, Unit

import zipfile
from zipfile import ZipFile
import io
import geopandas as gpd
import fiona
#import pyogrio 
import time
from time import sleep
from progress.bar import Bar
from numpy import interp

import sys,random

def progressBar(count_value, total, suffix=''):
    bar_length = 100
    filled_up_Length = int(round(bar_length* count_value / float(total)))
    percentage = round(100.0 * count_value/float(total),1)
    bar = '=' * filled_up_Length + '-' * (bar_length - filled_up_Length)
    sys.stdout.write('[%s] %s%s ...%s\r' %(bar, percentage, '%', suffix))
    sys.stdout.flush()
    
#    
class Progress:
    def __init__(self, value, end, title='Processing',buffer=50):
        self.title = title
        #when calling in a for loop it doesn't include the last number
        self.end = end -1
        self.buffer = buffer
        self.value = value
        self.progress()

    def progress(self):
        maped = int(interp(self.value, [0, self.end], [0, self.buffer]))
        #print(Fore.BLUE + '======> game-ply # ',count_game,plycount,' over ',plymax)     
        print(Fore.BLUE + f'{self.title}: [{"#"*maped}{"-"*(self.buffer - maped)}]{self.value}/{self.end} {((self.value/self.end)*100):.2f}%', end='\r')

    
def gini(x):
    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

def ginifast(x):
    # The rest of the code requires numpy arrays.
    #x = np.asarray(x)       
    sorted_x = np.sort(x)
    n = len(x)
    cumx = np.cumsum(sorted_x, dtype=float)
    # The above formula, with all weights equal to 1 simplifies to:
    return (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n
    
    
pd.set_option('display.max_rows', 100)
#ServerApp.iopub_data_rate_limit

%matplotlib inline
#ox.config(log_console=True)
ox.__version__
#nx.config(log_console=True)
nx.__version__

### init code
path = './dataverse_files/hong_kong-3d_gpkg.zip'
mylist = []
count = 0
    

#elev_all = []
#df_elev = pd.DataFrame()
#df_all = pd.DataFrame(columns=('City', 'Country', 'Gini', 'Min', 'Max','Mean','std','rel_std'))
#icount = 0

def func_elev(u):
    return z[u]
                
def func_trans(u):
    if u > 0:
        return u
    else:
        return 0.0
    
                
### start reading 
for filename in os.listdir(path):
    count += 1
    country = filename.split('-')[0].capitalize()
    mylist.append(filename)
    
    namefile = str(path+filename)
    print('---- namefile= ',namefile)
    
    list_elev = []
    list_ratl = []
    list_dd = []
    list_dz = []
    list_dz0 = []
    list_ratz = []
    list_ratz_flat = []
    list_ratbase = []
    
    with zipfile.ZipFile(namefile) as z:
        for filename2 in z.namelist():
            city = filename2.split('-')[0].capitalize()
            
            #filepath 
            fp =  "zip://"+str(namefile)+"!"+str(filename2)            
            print('fp layers= ',fiona.listlayers(fp))
            
            # read gpd
            gdf_nodes = gpd.read_file(fp, layer='nodes',encoding= 'unicode_escape').set_index('osmid')
            gdf_edges = gpd.read_file(fp, layer='edges',encoding= 'unicode_escape') #.set_index(['u', 'v', 'key'])
            assert gdf_nodes.index.is_unique and gdf_edges.index.is_unique
            
            z = gdf_nodes.to_dict(index=True)['elevation']
            x = gdf_nodes.to_dict(index=True)['x']
            y = gdf_nodes.to_dict(index=True)['y']
            
            dict_pos = {x[0]: (x[2],x[1]) for x in gdf_nodes.itertuples(index=True)}
            list_elev = gdf_nodes['elevation'].to_numpy()
            elevation = gdf_nodes['elevation'].to_numpy()
            gini_elev = ginifast(elevation)
            min_elev = np.min(elevation)
            max_elev = np.max(elevation)
            mean_elev = np.mean(elevation)
            std_elev = np.std(elevation)
            rel_std = std_elev/mean_elev
            print(city,country,'gini stat= ',gini_elev,min_elev,max_elev,mean_elev,std_elev,rel_std)
       
            index_nodes = list(gdf_nodes.index.values)
            
            G3d = nx.from_pandas_edgelist(gdf_edges, 'u', 'v',
                                    create_using=nx.Graph(), edge_attr='length_3d')
            
            G3d.remove_edges_from(nx.selfloop_edges(G3d))


            print('G3d node prop= ',G3d.number_of_nodes(),G3d.size())
            print('number of cc= ',nx.number_connected_components(G3d))
            largest_cc = max(nx.connected_components(G3d), key=len)
            lcc = len(largest_cc)
            print('size of largest ',lcc)
                 
            # BC computation node
            myk = lcc
            #myk = 2000
            print('compute BC node ',myk)
            start_time = time.time()
            bc_node = nx.betweenness_centrality(G3d, k=myk, weight="length_2d")
            ###########  for taking the elevation into account use:
            #bc_node = nx.betweenness_centrality(G3d, k=myk, weight="length_3d")
            print('time= ',time.time()-start_time)
            centrality_node = np.fromiter(bc_node.values(), float)
            max_node, max_bc = max(bc_node.items(), key=lambda x: x[1])
            min_node, min_bc = min(bc_node.items(), key=lambda x: x[1])
            print('min= ',min_node, min_bc)
            print('max= ',max_node, max_bc)
            print('len= ',len(bc_node))
            #print('bc node = ',bc_node)
            #print('nodes= ',sorted(G.node
    
            # print map
            fig, ax = plt.subplots(1, 1,figsize=(15,10)) 
        
            nx.draw(G3d, pos=dict_pos, edge_color='grey', ax=ax,node_color=centrality_node*100, width=1.0, node_size=centrality_node*5e3,alpha=0.3)

            fig.savefig('fig_new_BC_map_hk.pdf')
    
            ### 2nd figure
            fig2, ax2 = plt.subplots(1, 1,figsize=(15,10)) 
            sample_size = myk
            list1 = np.random.choice(index_nodes, size=sample_size)
            sample_frac = (float(sample_size)/float(lcc))**2
            print('relative size of sample= ',100.0*sample_frac,'%')

            list_elev = []
            list_bc = []
            for source in list1:
                list_elev.append(z[source])
                list_bc.append(bc_node[source])
                
            x_arr = np.array(list_elev)
            y_arr = np.array(list_bc)
            sns.scatterplot(x=x_arr, y=y_arr, ax=ax2, color='blue',alpha=0.5,s=100.0)#,legend=False)
            #sns.scatterplot(x=df_ard['dist'], y=df_ard['gain'], data=df_ard, ax=ax[2], color='green',alpha=0.6,s=100.0)
            ax2.set_xlabel(r'$z$',fontsize=24)
            ax2.set_ylabel(r'$BC$', fontsize=24)
            ax2.tick_params(axis='both', labelsize=20) 
            
            
            ### print average line
            df_ard = pd.DataFrame(columns=('z', 'bc'))
            df_ard['z'] = x_arr
            df_ard['bc'] = y_arr
            df_ardnew = df_ard.loc[df_ard['z'] > 0]      
            
            min_z = df_ard.min(axis=0)['z']
            max_z = df_ard.max(axis=0)['z']
            bins = np.linspace(min_z,max_z,30)
            group = df_ard.groupby(pd.cut(df_ard.z, bins))
            plot_centers = (bins[:-1]+bins[1:])/2
            plot_values = group.bc.mean()
            ax2.plot(plot_centers,plot_values, color='lime',alpha=1.0,linewidth=3)
            # without zeroes
            min_z2 = df_ardnew.min(axis=0)['z']
            max_z2 = df_ardnew.max(axis=0)['z']
            bins2 = np.linspace(min_z2,max_z2,30)
            group2 = df_ardnew.groupby(pd.cut(df_ardnew.z, bins2))
            #print('group= ',df_group)
            plot_centers2 = (bins2[:-1]+bins2[1:])/2
            plot_values2 = group2.bc.mean()
            ax2.plot(plot_centers2,plot_values2, color='crimson',alpha=1.0,linewidth=3)
            
            ax2.set(xscale='linear')
            ax2.set(yscale='linear')
        
            fig2.savefig('fig_BC_vs_z_sf.pdf')
        

print('number of files= ',count)